In [1]:
import cartopy.crs as ccrs
import rasterio
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from rasterio.plot import show

import folium
import geopandas as gpd

In [2]:
raster = rasterio.open('testTiff.tif')
lst = raster.read()
lst = lst.astype(float)
print(lst)

meta = raster.meta
meta.update(driver='GTiff')
meta.update(dtype=rasterio.float64)

with rasterio.open('lst.tif', 'w', **meta) as dst:
    dst.write(lst)

[[[    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  ...
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]]

 [[    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  ...
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]]

 [[    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  ...
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]]

 ...

 [[  255.   255.   255. ...   255.   255.   255.]
  [  255.   255.   2

In [3]:
#m = folium.Map(location=[35.9452,14.3797], zoom_start=10)
m = folium.Map(location=[14.3797,35.9452], zoom_start=10)

boundsdata = r'C:/Users/Massimo/Documents/SchoolFolder/ThirdYear/Thesis/MassThesis/src/References/malta.geojson'
region = gpd.read_file(boundsdata)
nReserve_proj = region.to_crs({'init': 'epsg:4326'})

C:\Users\Massimo\anaconda3\envs\snap\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [5]:
from rasterio.mask import mask

with rasterio.open("lst.tif") as src:
    out_image, out_transform = rasterio.mask.mask(src, nReserve_proj.geometry,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open('cropped.tif', 'w', **meta) as dst:
    dst.write(out_image)

## Working

In [27]:
import random
import rasterio
from rasterio.windows import Window

path_laptop = 'C:/Users/Massimo/Documents/SchoolFolder/ThirdYear/Thesis/SentinelData/TIF_files/S3B_SL_2_LST____20210616T094035_20210616T094335_20210617T190722_0179_053_307_2340_LN2_O_NT_004_LST.tif'

with rasterio.open(path_laptop) as src:

    # The size in pixels of your desired window
    #xsize, ysize = 512, 512
    xsize, ysize = 64, 64

    # Generate a random window location that doesn't go outside the image
    xmin, xmax = 0, src.width - xsize
    ymin, ymax = 0, src.height - ysize
    #xoff, yoff = random.randint(xmin, xmax), random.randint(ymin, ymax)
    xoff, yoff = 1270, 390

    # Create a Window and calculate the transform from the source dataset    
    window = Window(xoff, yoff, xsize, ysize)
    '''print('xoff: ' + str(xoff))
    print('yoff: ' + str(yoff))
    print('xsize: ' + str(xsize))
    print('ysize: ' + str(ysize))'''
    transform = src.window_transform(window)

    # Create a new cropped raster to write to
    profile = src.profile
    profile.update({
        'height': xsize,
        'width': ysize,
        'transform': transform})

    with rasterio.open('cropped.tif', 'w', **profile) as dst:
        # Read the data from the window and write it to the output raster
        dst.write(src.read(window=window))